<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
### Hidden Layer:
### Output Layer:
### Neuron:
### Weight:
### Activation Function:
### Node Map:
### Perceptron:


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [188]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score

np.random.seed(424)

data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

df

,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [189]:
##### Your Code Here #####
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

df['bias'] = np.ones((4,), dtype=int)
df

,x1,x2,y,bias
0,0,0,1,1
1,1,0,1,1
2,0,1,1,1
3,1,1,0,1


In [190]:
inputs = df[["x1", "x2", "bias"]].values
correct_outputs = [[value] for value in df["y"]]
weights = np.random.random((3,1))

In [191]:
inputs

array([[0, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 1]])

In [192]:
correct_outputs

[[1], [1], [1], [0]]

In [193]:
print("Initial weights\n", weights)

# Update our weights 10,000 times
for iteration in range(10000):
    
    # Weighted sum of inputs / weights
    weighted_sum = np.dot(inputs, weights)
    
    # Activate!
    activated_output = sigmoid(weighted_sum)
    
    # Cac error
    error = correct_outputs - activated_output
    
    adjustments = error * sigmoid_derivative(weighted_sum)
    
    # Update the Weights
    weights += np.dot(inputs.T, adjustments)
    
print("Weights after training")
print(weights)

print("Output after training")
print(activated_output)

Initial weights
 [[0.81625866]
 [0.46517734]
 [0.25836587]]
Weights after training
[[-8.01834065]
 [-8.01834065]
 [12.11397756]]
Output after training
[[0.99999452]
 [0.98362654]
 [0.98362654]
 [0.01940458]]


In [337]:
obs = 1000
choices = [1, 0]

def nand(val1, val2):
     return 0 if (val1 == 1) & (val2 == 1) else 1
    
x1 = [random.choice(choices) for i in range(0,obs)]
x2 = [random.choice(choices) for i in range(0,obs)]
y = []

for i in range(0, obs): 
    y_true = nand(x1[i], x2[i])
    y.append(y_true)
    
df_test = pd.DataFrame({'x1': x1, 'x2': x2, 'y':y})
df_test["bias"] = pd.Series(np.ones(obs,), dtype=int)
df_test

,x1,x2,y,bias
0,1,0,1,1
1,1,1,0,1
2,1,1,0,1
3,0,0,1,1
4,0,0,1,1
...,...,...,...,...
995,1,0,1,1
996,1,0,1,1
997,1,1,0,1
998,1,1,0,1


In [386]:
# Thanks Brian : https://github.com/BrianThomasRoss/lambda-school/blob/master/curriculum/unit-4-machine-learning/sprint-2-intro-nn/module1-Intro-to-Neural-Networks/LS_DS_421_Intro_to_NN_Assignment.ipynb

class Perceptron(object):
    
    def __init__(self, rate = .01, niter = 1000):
        self.rate = rate
        self.niter = niter
    
    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def fit(self, X, y):

        self.weight = 2 * np.random.random((1, X.shape[1])) - 1
        self.errors = [] 

        for i in range(self.niter):
            err = 0
            for xi, target in zip(X, y):
                pred = self.predict(xi)
                delta_w = self.rate * (target - pred) * self.sigmoid_derivative(xi)
                self.weight += delta_w * xi
                err += int(np.linalg.norm(delta_w) != 0.0)
            self.errors.append(err)
        return self

    def net_input(self, X):
        return np.dot(self.weight, X)

    def predict(self, X):
        return self.sigmoid(self.net_input(X.T))

In [339]:
pn = Perceptron()
pn.fit(df_test[['x1','x2', 'bias']].values, df_test['y'].values)

In [340]:
predictions = pn.predict(df_test[['x1','x2', 'bias']].values)

In [341]:
predictions = np.round(predictions)

In [342]:
test_y_labels = df_test["y"].tolist()

In [343]:
predictions = predictions[0].tolist()

In [344]:
accuracy_score(df_test["y"], predictions)

1.0

## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [345]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [375]:
diabetes["Bias"] = 1

diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Bias
0,6,148,72,35,0,33.6,0.627,50,1,1
1,1,85,66,29,0,26.6,0.351,31,0,1
2,8,183,64,0,0,23.3,0.672,32,1,1
3,1,89,66,23,94,28.1,0.167,21,0,1
4,0,137,40,35,168,43.1,2.288,33,1,1
...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,1
764,2,122,70,27,0,36.8,0.340,27,0,1
765,5,121,72,23,112,26.2,0.245,30,0,1
766,1,126,60,0,0,30.1,0.349,47,1,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [377]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes.drop(columns="Outcome"))
feats

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Bias']

In [378]:
X = Normalizer().fit_transform(diabetes[feats])
print(X.shape)
X

(768, 9)


array([[0.03355185, 0.82761219, 0.40262215, ..., 0.00350617, 0.27959871,
        0.00559197],
       [0.00842371, 0.71601493, 0.55596454, ..., 0.00295672, 0.26113486,
        0.00842371],
       [0.04039717, 0.9240852 , 0.32317734, ..., 0.00339336, 0.16158867,
        0.00504965],
       ...,
       [0.026915  , 0.65134299, 0.38757599, ..., 0.00131883, 0.16149   ,
        0.005383  ],
       [0.00665291, 0.83826692, 0.39917472, ..., 0.00232187, 0.31268687,
        0.00665291],
       [0.00791429, 0.73602906, 0.55400037, ..., 0.002493  , 0.18202869,
        0.00791429]])

In [379]:
y = diabetes["Outcome"].values
y

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,

In [426]:
per = Perceptron(5, 1000)
per.fit(X, y)

In [427]:
predictions = per.predict(X)
predictions = np.round(predictions)
predictions = predictions.tolist()[0]

In [428]:
diabetus_y_labels = y.tolist()

In [430]:
accuracy_score(diabetus_y_labels, predictions) * 100

69.27083333333334

In [432]:
(diabetes["Outcome"].value_counts(dropna=False, normalize=True) * 100)[0]

65.10416666666666

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?